In [5]:
import pickle

file_path = '../assets/db/db5.pkl'

with open(file_path, 'rb') as fp:
    data = pickle.load(fp)

data

{'2sAs3s5s8o': (68863, 26150),
 '2sAs3s4s6s': (94382, 2143),
 '2sAs3s5o5s': (67675, 25071),
 '2sAs3s4s9s': (94790, 1735),
 '2sAs3s5sTo': (67653, 26998),
 '2sAs3s5sKs': (95057, 1468),
 '2sAs3s4s9o': (66780, 28153),
 '2sAs3s5s6s': (94442, 1803),
 '2sAs3s4s8s': (94868, 1657),
 '2sAs3s5s9s': (94725, 1800),
 '2sAs3s4sTs': (94667, 1858),
 '2sAs3s5s7s': (94447, 2078),
 '2sAs3s5s6o': (60149, 33277),
 '2sAs3s4sKs': (94921, 1604),
 '2sAs3s4o5s': (82683, 8092),
 '2sAs3s4s5o': (83739, 7674),
 '2sAs3s4s7s': (94598, 1927),
 '2sAs2o3s4s': (69643, 23801),
 '2sAs3s4s8o': (63751, 31270),
 '2sAs3s4sQo': (65269, 29643),
 '2sAs3s4s7o': (63291, 31969),
 '2sAs3s5sJs': (94933, 1592),
 '2sAs3s5s9o': (65178, 29428),
 '2sAs3s5sQs': (94961, 1564),
 '2sAs3s4s5s': (96142, 383),
 '2sAs3s4sTo': (65667, 29267),
 '2sAs3s5sTs': (94833, 1692),
 '2sAs3s4sJo': (66405, 28463),
 '2sAs3s4sQs': (94883, 1642),
 '2sAs3s4sJs': (95118, 1407),
 '2sAs3s4sAo': (84129, 9742),
 '2sAs3o3s5s': (68467, 24250),
 '2sAs3o3s4s': (69592, 23317

In [ ]:
NUM_HANDS_1 = 96525
NUM_HANDS_2 = 45540
NUM_HANDS_3 = 990